In [358]:
%%capture

!pip install Spotipy dataprep

#### Imports and Connecting to the API

In [114]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import pandas as pd
from dataprep.eda import plot, plot_correlation, plot_missing

client_credentials_manager = SpotifyClientCredentials(client_id='f13a6b55c2d949939455644bbd6f31d9', client_secret='c7c009270ff0483db51d25543563d55c')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

### Loading a playlist

In [360]:
playlist = sp.playlist('open.spotify.com/playlist/2YRe7HRKNRvXdJBp9nXFza')

In [108]:
# dictionary of information on a trakc - URI can be used to pull further info

dict(song=playlist['tracks']['items'][1]['track']['name'], 
     artist=playlist['tracks']['items'][1]['track']['album']['artists'][0]['name'],
     uri=playlist['tracks']['items'][1]['track']['uri'])

{'song': 'rockstar (feat. 21 Savage)',
 'artist': 'Post Malone',
 'uri': 'spotify:track:0e7ipj03S05BNilyu5bRzt'}

In [158]:
# list of songs in playlist - title, artist, uri, popularity

top_100 = []
for i in range(0, len(playlist['tracks']['items'])):
    top_100.append(dict(song=playlist['tracks']['items'][i]['track']['name'], 
                        artist=playlist['tracks']['items'][i]['track']['album']['artists'][0]['name'],
                        uri=playlist['tracks']['items'][i]['track']['uri'],
                        popularity=playlist['tracks']['items'][i]['track']['popularity']))

In [147]:
features = []
names = []

# pulling audio features from api by track uri

for i in range(0, len(top_100)):
    song_features = sp.audio_features(top_100[i]['uri'])
    for track in song_features:
        features.append(track)
        
# list of names for indexing

for i in range(0, len(top_100)):
    song_name = top_100[i]['song']
    names.append(song_name)

In [ ]:
### DF of song features

In [174]:
song_features = pd.DataFrame(features, index = names)

In [361]:
# popularity ranking of song

popularity = [top_100[i]['popularity'] for i in range(0, len(top_100))]

song_features['popularity'] = popularity

In [373]:
# sorting by popularity

sf_sorted = song_features.sort_values(by='popularity', ascending=False)

In [374]:
sf_sorted = sf_sorted.drop(labels=['type', 'id', 'uri', 'track_href', 'analysis_url'], axis=1)

In [375]:
sf_sorted.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,popularity
bad guy,0.701,0.425,7,-10.965,1,0.3750,0.328,0.130000,0.100,0.562,135.128,194088,4,93
Someone You Loved,0.501,0.405,1,-5.679,1,0.0319,0.751,0.000000,0.105,0.446,109.891,182161,4,93
lovely (with Khalid),0.351,0.296,4,-10.109,0,0.0333,0.934,0.000000,0.095,0.120,115.284,200186,4,91
Jocelyn Flores,0.872,0.391,0,-9.144,0,0.2420,0.469,0.000004,0.297,0.437,134.021,119133,4,88
rockstar (feat. 21 Savage),0.585,0.520,5,-6.136,0,0.0712,0.124,0.000070,0.131,0.129,159.801,218147,4,88


### Quick EDA of audio features

In [376]:
plot(sf_sorted)

In [377]:
plot(sf_sorted, "speechiness", bins=20)

In [378]:
plot(sf_sorted, "liveness", bins=20)

In [379]:
plot(sf_sorted, "danceability", bins=20)

In [380]:
plot(sf_sorted, "energy", bins=20)

In [382]:
plot(sf_sorted, 'energy', 'danceability', bins=20)

In [383]:
plot_correlation(sf_sorted, k=5)

In [384]:
plot_correlation(sf_sorted, 'danceability')

In [385]:
plot_correlation(sf_sorted, x='valence', y='energy')